In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import json

from unicode import split_syllables, join_jamos
from tqdm import tqdm

# Method 1

## Data Load

In [2]:
text = "text"

In [3]:
all_data = load_dataset('csv',data_files='./order_speech_ko.csv',split='train')

Using custom data configuration default-8e0d0b8910acc620
Reusing dataset csv (C:\Users\Mu-jun\.cache\huggingface\datasets\csv\default-8e0d0b8910acc620\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


## Data Cleaning

In [4]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch[text] = re.sub(chars_to_ignore_regex, '', batch[text]).lower() + " "
    return batch

In [5]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x000001FE3DF02EE8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/142367 [00:00<?, ?ex/s]

## Split to Font

In [6]:
target_data = remove_spectial_char_data
jamos = []
for i in tqdm(range(len(target_data))):
    split_text = split_syllables(target_data[i][text])
    all_text = " ".join(split_text)
    jamos.extend(list(set(all_text)))

100%|██████████| 142367/142367 [00:26<00:00, 5462.40it/s]


## Add index

In [7]:
vocab_list = list(set(jamos))

In [8]:
vocab_dict = {v: k+5 for k, v in enumerate(vocab_list)}

## Add Special Token

In [9]:
vocab_dict["|"] = 4
vocab_dict["<pad>"] = 0
vocab_dict["<s>"] = 1
vocab_dict["</s>"] = 2
vocab_dict["<unk>"] = 3
len(vocab_dict)

60

In [10]:
vocab_dict

{'ㅖ': 5,
 'ㅙ': 6,
 'ㄸ': 7,
 'ㄿ': 8,
 'ㅅ': 9,
 'ㅝ': 10,
 'ㅏ': 11,
 't': 12,
 'ㅜ': 13,
 'ㄹ': 14,
 'ㅛ': 15,
 'ㅘ': 16,
 'ㅡ': 17,
 'ㅐ': 18,
 'ㄲ': 19,
 'ㄷ': 20,
 'ㄺ': 21,
 'ㅒ': 22,
 'ㅎ': 23,
 'c': 24,
 'ㄴ': 25,
 'ㅁ': 26,
 'ㅉ': 27,
 'ㅊ': 28,
 'ㅈ': 29,
 'ㄱ': 30,
 'ㅄ': 31,
 'ㄶ': 32,
 '5': 33,
 '3': 34,
 "'": 35,
 'm': 36,
 'ㅃ': 37,
 'ㅀ': 38,
 'ㅓ': 39,
 'ㄼ': 40,
 'ㅚ': 41,
 'ㅠ': 42,
 'ㅑ': 43,
 'ㅇ': 44,
 'ㅆ': 45,
 'ㅕ': 46,
 'ㅋ': 47,
 'ㅍ': 48,
 'v': 49,
 'ㅣ': 50,
 'ㅂ': 51,
 'ㅢ': 52,
 'b': 53,
 ' ': 54,
 'ㅔ': 55,
 'ㅟ': 56,
 '2': 57,
 'ㅗ': 58,
 'ㅌ': 59,
 '|': 4,
 '<pad>': 0,
 '<s>': 1,
 '</s>': 2,
 '<unk>': 3}

## Export Vocabulary to json file

In [11]:
with open('vocab_jamos.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Method 2

In [2]:
print('ㄱ~ㅎ ->',ord('ㄱ'),'~',ord('ㅎ'))

ㄱ~ㅎ -> 12593 ~ 12622


In [3]:
print('ㅏ~ㅣ->',ord('ㅏ'),'~',ord('ㅣ'))

ㅏ~ㅣ-> 12623 ~ 12643


In [4]:
print('가~힣 ->',ord('가'),'~',ord('힣'))

가~힣 -> 44032 ~ 55203


In [3]:
def char_list(start_chr_num, end_chr_num):
    return [chr(i) for i in range(start_chr_num,end_chr_num+1)]

In [13]:
jamos = []
jamos.extend(char_list(ord('ㄱ'), ord('ㅎ')))
print(len(jamos))

jamos.extend(char_list(ord('ㅏ'), ord('ㅣ')))
print(len(jamos))

jamos.extend(char_list(ord('a'), ord('z')))
print(len(jamos))

jamos.extend(char_list(ord('A'), ord('Z')))
print(len(jamos))

print()

chars = jamos.copy()
chars.extend(char_list(ord('가'), ord('힣')))
print(len(chars))

30
51
77
103

11275


In [5]:
from unicode import split_syllables, join_jamos

In [6]:
split_syllables('<unk>')

'<unk>'

In [7]:
vocab_list = list(set(jamos))
len(vocab_list)

103

In [8]:
vocab_dict = {v: k+8 for k, v in enumerate(vocab_list)}

In [9]:
vocab_dict["<pad>"] = 0
vocab_dict["<s>"] = 1
vocab_dict["</s>"] = 2
vocab_dict["<unk>"] = 3
vocab_dict["|"] = 4
vocab_dict["<b>"] = 5
vocab_dict["<n>"] = 6
vocab_dict["<l>"] = 7
len(vocab_dict)

111

In [10]:
with open('vocab_jamos.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [11]:
vocab_list = list(set(chars))
vocab_dict = {v: k+8 for k, v in enumerate(vocab_list)}
vocab_dict["<pad>"] = 0
vocab_dict["<s>"] = 1
vocab_dict["</s>"] = 2
vocab_dict["<unk>"] = 3
vocab_dict["|"] = 4
vocab_dict["<b>"] = 5
vocab_dict["<n>"] = 6
vocab_dict["<l>"] = 7
len(vocab_dict)

11232

In [12]:
with open('vocab_chars.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)